# CS 109A/AC 209A/STAT 121A Data Science: Final Project
**Harvard University**<br>
**Fall 2016**<br>
**Instructors:** W. Pan, P. Protopapas, K. Rader<br>
**Members: ** Shawn Pan, Xinyuan (Amy) Wang, Ming-long Wu

## SVR ##
- $R^2 = 0.678$
- To slow to be a practical model, and performs the same as random forest. 

In [1]:
import numpy as np
import pandas as pd
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import time
from itertools import combinations
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
%matplotlib inline

In [2]:
# Load listing data
listing = pd.read_csv('../../Data/data_after_predictor_selection.csv')
listing.head(2)

,accommodates,bathrooms,bedrooms,review_scores_checkin,review_scores_communication,latitude,longitude,property_type_0,property_type_1,property_type_2,...,bed_type_2,bed_type_3,bed_type_4,beds,review_scores_value,host_listing_count,review_scores_cleanliness,review_scores_accuracy,minimum_nights,price
0,2,1.0,1.0,9.0,9.0,40.756852,-73.964754,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,9.0,1,7.0,9.0,3,160
1,10,1.0,3.0,9.0,8.0,40.830599,-73.941014,1.0,0.0,0.0,...,0.0,0.0,0.0,3.0,8.0,2,8.0,8.0,1,105


In [3]:
y = np.log(listing["price"])
x = listing[[col for col in listing.columns if col != "price"]]
print x.shape
print y.shape

(27392, 27)
(27392,)


In [4]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.neighbors import KNeighborsRegressor

class LocationPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = KNeighborsRegressor(n_neighbors=50, weights="uniform")
        super(LocationPreprocessor, self).__init__()
        
    def fit_transform(self, X, y):
        self.model.fit(X[["longitude", "latitude"]], y)
        return self.transform(X)
    
    def transform(self, X):
        loc = self.model.predict(X[["longitude", "latitude"]])
        Xnew = X.copy()
        Xnew["location"] = loc
        return Xnew

# SVR

### Single Split at Various C 
(too slow to do full grid search)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
model = Pipeline([
            ("location", LocationPreprocessor()),
            ("svr", SVR(C=1))
        ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
print "Compute Time", compute_time

Train Score 0.706609314812
Test Score 0.676630285219
Compute Time 16.7144479752


In [6]:
model = Pipeline([
            ("location", LocationPreprocessor()),
            ("svr", SVR(C=10))
        ])

start_time = time.time()
model.fit(x_train, y_train)
compute_time = time.time() - start_time

print "Train Score", model.score(x_train, y_train)
print "Test Score", model.score(x_test, y_test)
print "Compute Time", compute_time

Train Score 0.735636620129
Test Score 0.678170699523
Compute Time 35.0556018353


### Tuning by Cross-Validation
too slow be practical

In [ ]:
#too slow
params = {"svr__C": [1, 5, 20],
          "svr__kernel": ["rbf", "linear"]}

gs = GridSearchCV(model, param_grid=params, n_jobs=10)
start_time = time.time()
gs.fit(x, y);
compute_time = time.time() - start_time

print gs.best_score_
print gs.best_params_
print "Compute Time", compute_time

### SVR is too slow to be practical!